In [57]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [70]:
## Load dataset
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [71]:
data.shape

(10000, 14)

In [72]:
## Preprocess Data
## Drop irrelevant features

data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [73]:
## Missing values?
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  int64  
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(2), int64(7), object(2)
memory usage: 859.5+ KB


In [74]:
## Encode Features : Categorical Features
## 1. Gender
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [75]:
## In Case of Geography column: if we use label encoder then 
#   ANN will consider the category greater which is encoded with higher value of number, And IT IS A MAJOR ISSUE

## We will do One Hot Encoding
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo = OneHotEncoder()
geo_encoder = onehot_encoder_geo.fit_transform(data[['Geography']])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [76]:
onehot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [77]:
## Assign encoded Geography to dataframe
geo_encoded_df = pd.DataFrame(geo_encoder.toarray(), columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [78]:
## Combine onehot_encoded columns to data
data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [79]:
##  save the encoder and sscaler
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)

In [80]:
## Divide the dataset into dependent and independent features
X = data.drop('Exited', axis=1)
y = data['Exited']

## Split the data into training and test set
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2,random_state=42)

## Scale down the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [81]:
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [97]:
## Save the scaler
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)


False

## ANN Implementation

In [83]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [84]:
X_train.shape

(8000, 12)

In [85]:
## Build ANN model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)), ## HiddenLayer 1 = Connected with InputLayer
    Dense(32, activation='relu'), ## HiddenLayer 2
    Dense(1, activation='sigmoid') ## Output Layer
])

In [86]:
## See how many total parameters our model has
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 64)                832       
                                                                 
 dense_7 (Dense)             (None, 32)                2080      
                                                                 
 dense_8 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [87]:
# One way to initialize optimizer and loss from tensorflow
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01) ## in this optimizer we can provide out own learning rate
loss = tf.keras.losses.BinaryCrossentropy()

In [88]:
## compile the model : to do forward and backward propagation

## Another way: to use string value to initialize directly while compiling model
# model.compile(optimizer="adam", loss="binary_crossentropy", metrics=['accuracy']) # here optimizer="adam" has fixed learning rate

model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=['accuracy'])

In [89]:
## Setup the Tensorboard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") # Create logs directory
tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1) ## to visualize all the logs while training the model, can be achieved with matplotlib as well


In [90]:
## Setup Early Stopping
# we can train the model for any number of epochs, and if our model while training for sometime has negligible change in loss then it is not necessory to train that model further
#   In that case we can apply early stopping to stop the training

early_stopping_callback = EarlyStopping(monitor='val_loss', patience=60, restore_best_weights=True) #patience=5 : weight till 5 epochs to stop if no change in loss


In [91]:
## Train the model
history = model.fit(
    X_train,y_train, validation_data=(X_test, y_test), epochs=100,
    callbacks=[tensorflow_callback, early_stopping_callback]
)

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3920 - accuracy: 0.8346 - val_loss: 0.3544 - val_accuracy: 0.8545
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3557 - accuracy: 0.8608 - val_loss: 0.3557 - val_accuracy: 0.8625
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3468 - accuracy: 0.8589 - val_loss: 0.3438 - val_accuracy: 0.8625
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3450 - accuracy: 0.8575 - val_loss: 0.3623 - val_accuracy: 0.8545
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3408 - accuracy: 0.8616 - val_loss: 0.3506 - val_accuracy: 0.8575
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3386 - accuracy: 0.8602 - val_loss: 0.3574 - val_accuracy: 0.8560
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3359 - accuracy: 0.8604 - val_loss: 0.3435 - val_accuracy: 0.8585

In [92]:
model.save('model.h5') ## h5 file is compatilbe with keras

c:\NLP_PROJECTS\classification_ann\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [93]:
## Load TensorBoard Extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [94]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 11560), started 1:38:07 ago. (Use '!kill 11560' to kill it.)